# 227. Basic Calculator II

https://leetcode.com/problems/basic-calculator-ii/

In [ ]:
class Solution:
    def calculate(self, s: str) -> int:
        tokens = []
        
        #print(re.split('[-+*/]', s))
        operator = '+-/*'
        
        index = 0
        digits = 0
        value = ''

        for i in range(len(s)):
            if s[i] in operator:
                tokens.append(int(value))
                value = ''
                tokens.append(s[i])
            else:
                if s[i] != ' ':
                    value +=  s[i] 
                
        tokens.append(int(value)) 
    
        # two passes?
        new_tokens = []
        value = None
        i = 0
        while i < len(tokens):
            
            if tokens[i] == '*':
                i+= 1
                new_tokens[-1] = new_tokens[-1] * tokens[i]
            elif tokens[i] == '/':
                i += 1
                new_tokens[-1] = new_tokens[-1] // tokens[i]
            else: 
                new_tokens.append(tokens[i])
            i += 1
            
        final_value = new_tokens[0]
        i = 1
        while i< len( new_tokens):
            if new_tokens[i] == '+':
                i+=1
                final_value += new_tokens[i]
            elif new_tokens[i] == '-':
                i+=1
                final_value -= new_tokens[i]
            i+=1
        return final_value
        


# 85. Maximal Rectangle

https://leetcode.com/problems/maximal-rectangle/

Given a 2D binary matrix filled with 0's and 1's, find the largest rectangle containing only 1's and return its area.

Example:

    Input:
    [
      ["1","0","1","0","0"],
      ["1","0","1","1","1"],
      ["1","1","1","1","1"],
      ["1","0","0","1","0"]
    ]
    Output: 6


In [ ]:
class Solution:
    def maximalRectangle(self, matrix: List[List[str]]) -> int:
        if not matrix:
            return 0
        m, n = len(matrix), len(matrix[0])

        
        dp = [[0] * (n) for _ in range(m) ]

        max_area = 0

        for i in range(m):
            for j in range(n):
                if matrix[i][j]== "1":
                    if j ==0:
                        dp[i][j]=1
                    else:
                        dp[i][j] = dp[i][j-1] +1
                
                height = 0
                width = float("inf")
                for k in range(i, -1, -1):
                    height+= 1
                    width = min(width, dp[k][j])
                    if width == 0: # Early stopping?
                        break
                    area = height * width
                    max_area = max(area, max_area)
        
        return max_area



# 68. Text Justification

https://leetcode.com/problems/text-justification/

Given an array of words and a width maxWidth, format the text such that each line has exactly maxWidth characters and is fully (left and right) justified.

You should pack your words in a greedy approach; that is, pack as many words as you can in each line. Pad extra spaces ' ' when necessary so that each line has exactly maxWidth characters.

Extra spaces between words should be distributed as evenly as possible. If the number of spaces on a line do not divide evenly between words, the empty slots on the left will be assigned more spaces than the slots on the right.

For the last line of text, it should be left justified and no extra space is inserted between words.

Note:

- A word is defined as a character sequence consisting of non-space characters only.
- Each word's length is guaranteed to be greater than 0 and not exceed maxWidth.
- The input array words contains at least one word.

Example 1:

    Input:
    words = ["This", "is", "an", "example", "of", "text", "justification."]
    maxWidth = 16
    Output:
    [
       "This    is    an",
       "example  of text",
       "justification.  "
    ]

Example 2:

    Input:
    words = ["What","must","be","acknowledgment","shall","be"]
    maxWidth = 16
    Output:
    [
      "What   must   be",
      "acknowledgment  ",
      "shall be        "
    ]
    Explanation: Note that the last line is "shall be    " instead of "shall     be",
             because the last line must be left-justified instead of fully-justified.
             Note that the second line is also left-justified becase it contains only one word.
             
Example 3:

    Input:
    words = ["Science","is","what","we","understand","well","enough","to","explain",
             "to","a","computer.","Art","is","everything","else","we","do"]
    maxWidth = 20
    Output:
    [
      "Science  is  what we",
      "understand      well",
      "enough to explain to",
      "a  computer.  Art is",
      "everything  else  we",
      "do                  "
    ]


In [ ]:
class Solution:
    def fullJustify(self, words: List[str], maxWidth: int) -> List[str]:
        output = []
        current_line = []
        current_len = 0
        
        while words:
            this = words.pop(0)
            
            if (current_len + len(this)) <= maxWidth:
                current_line.append(this)
                current_len += (len(this)+1)
            else:
                output.append(current_line)
                current_line = [this]
                current_len = len(this)+1
                
        output.append(current_line)

        str_output = []
        for i in range(len(output)):
            if i == len(output)-1:
                last_line = True
            else: 
                last_line = False
            line = output[i]
            n_chars = sum([len(x) for x in line])
            n_spaces = len(line) - 1
            n_whitespace = maxWidth - n_chars
            
            str_line = ""
            if last_line:
                str_line = " ".join(line)
                str_line += " " * (maxWidth - len(str_line))
                
            elif len(line) > 1:
                count = 0
                for i in range(n_whitespace):
                    line[count % n_spaces]+= " "
                    count += 1
                    
                for i in line:
                    str_line += i
            else:
                str_line = line[0] + " " * n_whitespace #(maxWidth - len(line[0]))
            str_output.append(str_line) #avg_space = maxWidth - n_chars
            
        for line in str_output:
            print(len(line), line)
        return(str_output)
                
            

# Word Search II


Given a 2D board and a list of words from the dictionary, find all words in the board.

Each word must be constructed from letters of sequentially adjacent cell, where "adjacent" cells are those horizontally or vertically neighboring. The same letter cell may not be used more than once in a word.

 

Example:

    Input: 
    board = [
      ['o','a','a','n'],
      ['e','t','a','e'],
      ['i','h','k','r'],
      ['i','f','l','v']
    ]

    words = ["oath","pea","eat","rain"]

    Output: ["eat","oath"]
 

Note:

- All inputs are consist of lowercase letters a-z.
- The values of words are distinct.


In [ ]:
class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        d = self.build_not_a_trie(words)
        
        m, n = len(board), len(board[0])
        output = set()
        def helper(x,y, word_fragment, visited):
            if visited[x][y] == 1:
                return

            
            visited[x][y] = 1
            word_fragment += board[x][y]
            
            if word_fragment in words:
                output.add(word_fragment)
            if word_fragment in d.keys():
                if x>0:
                    helper(x-1,y, word_fragment, visited)
                if y>0:
                    helper(x,y-1, word_fragment, visited)
                if x<(m-1):
                    helper(x+1,y, word_fragment, visited)
                if y<(n-1):
                    helper(x,y+1, word_fragment, visited)
            visited[x][y] = 0

        for i in range(m):
            for j in range(n):
                visited =  [[0]*n for _ in range(m)] #visited_base.copy()
                helper(i,j, '', visited)
                
        return list(output)
    
        
    def build_not_a_trie(self, words):
        d = defaultdict(set)
        for word in words:
            for i in range(1,len(word)):
                d[word[0:i]].add(word)
        return d

In [ ]:
# Note that the solution had this handy "make a trie"
# that instead of using classes, nests dictionaries

        WORD_KEY = '$'
        trie = {}
        for word in words:
            node = trie
            for letter in word:
                # retrieve the next node; If not found, create a empty node.
                node = node.setdefault(letter, {})
            # mark the existence of a word in trie node
            node[WORD_KEY] = word
        print(trie)
        

#  Word Squares

Given a set of words (without duplicates), find all word squares you can build from them.

A sequence of words forms a valid word square if the kth row and column read the exact same string, where 0 ≤ k < max(numRows, numColumns).

For example, the word sequence ["ball","area","lead","lady"] forms a word square because each word reads the same both horizontally and vertically.

    b a l l
    a r e a
    l e a d
    l a d y
    
Note:
- There are at least 1 and at most 1000 words.
- All words will have the exact same length.
- Word length is at least 1 and at most 5.
- Each word contains only lowercase English alphabet a-z.

    Example 1:

    Input:
    ["area","lead","wall","lady","ball"]

    Output:
    [
      [ "wall",
        "area",
        "lead",
        "lady"
      ],
      [ "ball",
        "area",
        "lead",
        "lady"
      ]
    ]

    Explanation:
    The output consists of two word squares. The order of output does not matter (just the order of words in each word square matters).

Example 2:

    Input:
    ["abat","baba","atan","atal"]

    Output:
    [
      [ "baba",
        "abat",
        "baba",
        "atan"
      ],
      [ "baba",
        "abat",
        "baba",
        "atal"
      ]
    ]

    Explanation:
    The output consists of two word squares. The order of output does not matter (just the order of words in each word square matters).


In [ ]:
# First attempt: Seems to get a time limit exceeded, but otherwise seems to work

class Solution:
    def wordSquares(self, words: List[str]) -> List[List[str]]:
        self.squares = [] #store squares in here if recursive loop finds valid ones
        self.word_size = len(words[0])
        
        for word in words:
            # Adding a single word adds a constraint:
            self.helper(words, [word])
        return self.squares
        
    def helper(self, words, square):
        n_elems = len(square)
        if n_elems == self.word_size:
            self.squares.append(square)
            return True
        
        # We're now checking for the n_elems-indexed 
        # call function if a word matches condition
        for word in words:
            is_valid = True
            for i in range(n_elems):
                if square[i][n_elems] != word[i]:
                    is_valid = False
            if is_valid:
                new_square = square.copy()
                new_square.append(word)
                self.helper(words, new_square)
                


In [ ]:
# Second attempt: build a hash set of all prefixes, 
# then iterate through this!  Some more work at the beginning saves a lot of work later

class Solution:
    def wordSquares(self, words: List[str]) -> List[List[str]]:
        self.squares = [] #store squares in here if recursive loop finds valid ones
        self.word_size = len(words[0])
        self.build_prefix_hash(words)
        
        for word in words:
            # Adding a single word adds a constraint:
            self.helper(words, [word])
        return self.squares
    
    def build_prefix_hash(self,words):
        # Thought: a Trie works too, could it save memory?
        self.prefix_hash = defaultdict(list)
        for word in words:
            for i in range(1, self.word_size):
                self.prefix_hash[word[0:i]].append(word)

        
    def helper(self, words, square):
        n_elems = len(square)
        
        #base case: assume that if done correctly the square should be valid
        if n_elems == self.word_size:
            self.squares.append(square)
            return True
        prefix = ''.join([x[n_elems] for x in square])
        
        for word in self.prefix_hash[prefix]:
            new_square = square.copy()
            new_square.append(word)
            self.helper(words, new_square)
            
            

# Split Array Largest Sum


Given an array which consists of non-negative integers and an integer m, you can split the array into m non-empty continuous subarrays. Write an algorithm to minimize the largest sum among these m subarrays.

Note:
- If n is the length of array, assume the following constraints are satisfied:
- 1 ≤ n ≤ 1000
- 1 ≤ m ≤ min(50, n)

Examples:

    Input:
    nums = [7,2,5,10,8]
    m = 2

    Output:
    18

    Explanation:
    There are four ways to split nums into two subarrays.
    The best way is to split it into [7,2,5] and [10,8],
    where the largest sum among the two subarrays is only 18.


*There are two ways to do this, DP and binary sorting *

In [ ]:
class Solution:
    def splitArray(self, nums: List[int], m: int) -> int:
        #
        # For nums, consider dp[i][j] be the optimal result of
        # splitting nums[0:i] into j subarrays
        # 
        # Consider the last subarray j that is split from k to i.
        # dp[i][j], the max array, can be calculated by 
        # calculating the maximum of either dp[k][j-1] 
        # and the sum(nums[k+1:i])
        # 
        # We must try and minimize 
        #   max(dp[k][j-1], sum(nums[k+1:i]) )
        #
        # and stash the results in a matrix
        
        n_elems = len(nums)
        max_value = float(inf)
        dp = [[max_value for _ in range(n_elems+1)] for _ in range(m+1)]

        
        # To quickly find the sum of any range of N, one can 
        # just take the difference between two indices of 
        # the cumulative array
        cumulative = [0] * (n_elems+1)
        cumulative[0] = nums[0]
        for i in range(0,n_elems):
            cumulative[i+1] = cumulative[i] + nums[i]
        
        dp[0][0] = 0
        for i in range(1,n_elems+1):
            for j in range(1,m+1):
                for k in range(i):

                    dp[j][i] = min(dp[j][i], max(dp[j-1][k],cumulative[i]-cumulative[k]))
        return(dp[m][n_elems])



# Trapping Rain Water

link: https://leetcode.com/problems/trapping-rain-water/

Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it is able to trap after raining.


The above elevation map is represented by array [0,1,0,2,1,0,1,3,2,1,2,1]. In this case, 6 units of rain water (blue section) are being trapped. Thanks Marcos for contributing this image!

Example:

    Input: [0,1,0,2,1,0,1,3,2,1,2,1]
    Output: 6



In [ ]:
# First pass: works but brute force. After traversing array a lot
# can probably be sped up by memory

class Solution:
    def trap(self, height: List[int]) -> int:
        if not height:
            return 0 
        
        lhs = height[0]
        n_elems = len(height)
        volume = 0
        for i, val in enumerate(height):
            if val > lhs:
                # if this is iterated, assume that water can be trapped to the left
                lhs = val
                
            else: 
                # find rightmost wall
                rhs = val
                for j in range(i,n_elems):
                    if height[j] > rhs:
                        rhs = height[j]
                    if height[j] > lhs:
                        rhs = lhs
                        continue
                
                volume += min(lhs, rhs) - val
                
        return volume
                    

In [ ]:
# Second pass: use memory to do it in O(3N) = O(N) time, 
# instead of O(N^2) above
class Solution:
    def trap(self, height: List[int]) -> int:
        if not height:
            return 0 
        
        n_elems = len(height)
        volume = 0
        max_left = [0 for _ in height]
        max_right = [0 for _ in height]
        lhs = 0
        for i in range(n_elems):
            if height[i] > lhs:
                lhs = height[i]
            max_left[i] = lhs
        
        rhs = 0
        for i in range(n_elems-1, -1, -1):
            if height[i] > rhs:
                rhs = height[i]
            max_right[i] = rhs
            
        for i in range(n_elems):
            volume += min(max_left[i], max_right[i]) - height[i]
        
        return volume            
        

https://leetcode.com/problems/critical-connections-in-a-network/



In [ ]:

class Solution:
    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:
        
        # Create a dictionary where each node lists
        # the connecting nodes
        self.graph = defaultdict(list)
        for edge in connections:
            self.graph[edge[0]].append(edge[1])
            self.graph[edge[1]].append(edge[0])
        
        self.connections = [sorted(x) for x in connections]
        self.n = n
        
        #Start at node 0 with rank 0
        self.visited = {} #keep track of which nodes are visited, and at what rank they were in
        self.dfs(0, 0) 
        
        return self.connections
        
    def dfs(self,node, rank):
        if node in self.visited.keys(): #if we have previously visited it
            return self.visited[node]
        else:
            #first rank visited, most recent rank
            self.visited[node] = rank
            min_rank = self.n
            
            for neighbor in self.graph[node]:
                if (neighbor in self.visited.keys()) and (self.visited[neighbor] == rank-1):
                    continue
                    
                backwards_rank = self.dfs(neighbor, rank+1)
                if backwards_rank<=rank:
                    
                    self_connections = self.connections.remove(sorted([node, neighbor]))
                min_rank = min(min_rank, backwards_rank)
                
            return min_rank
                
                
                

In [ ]:

class Solution:
    def criticalConnections(self, n: int, connections: List[List[int]]) -> List[List[int]]:
        
        # Create a dictionary where each node lists
        # the connecting nodes
        self.graph = defaultdict(list)
        for edge in connections:
            self.graph[edge[0]].append(edge[1])
            self.graph[edge[1]].append(edge[0])
        
        self.connections = [sorted(x) for x in connections]
        self.n = n
        
        #self.visited = [False for i in range(n)]
        self.rank_number = [None for i in range(n)]
        ##keep track of which nodes are visited, and at what rank they were in
        
        #Start at node 0 with rank 0        
        self.dfs(0, 0) 
        
        return self.connections
        
    def dfs(self,node, rank):
        if not self.rank_number[node]: #if we have previously visited it
            #first rank visited, most recent rank
            self.rank_number[node] = rank
            
            min_rank = self.n
            
            for neighbor in self.graph[node]:
                if  self.rank_number[neighbor] == (rank-1):
                    continue
                    
                backwards_rank = self.dfs(neighbor, rank+1)
                if backwards_rank<=rank:
                    self_connections = self.connections.remove(sorted([node, neighbor]))
                min_rank = min(min_rank, backwards_rank)
            return min_rank
                
        else:
            # If we see a rank_number for this node, we can assume we visited
            # it before and return that rank number
            return self.rank_number[node] 

                